In [0]:
from preprocess import read_preprocessed_file, read_vocab
from collections import Counter
from scipy.stats import entropy
from numpy.linalg import norm
import os
import numpy as np
import spacy

en = spacy.load('en')

#yusu added words from the rule file
DEFAULT_MALE_NOUNS = {
    'Countryman',
    'fraternal','wizards','manservant','fathers','divo','actor','bachelor','papa',
    'dukes','barman','countrymen','brideprice','hosts','potential_suitors','airmen','andropause',
    'penis','prince','governors','abbot','men','widower','gentlemen','sorcerers','sir',
    'bridegrooms','baron','househusbands','gods','nephew','widowers','lord','brother',
    'grooms','priest','adultors','andrology','bellboys','his','marquis','princes','emperors',
    'stallion','chairman','monastery','priests','boyhood','fellas','king','dudes',
    'daddies','manservant','semen','spokesman','tailor','cowboys','dude','bachelors','barbershop','emperor','daddy',
    'masculism','guys','enchanter','guy','fatherhood','androgen','cameramen','godfather',
    'strongman','god','patriarch','uncle','chairmen','sir','brotherhood','host','testosterone',
    'husband','dad','steward','males','cialis','spokesmen','pa','beau','stud','bachelor',
    'wizard','sir','nephews','fathered','bull','beaus','councilmen','landladies','grandson',
    'fiances','stepfathers','horsewomen','grandfathers','adultor','schoolboy','rooster','grandsons',
    'bachelor','cameraman','dads','him','master','lad','policeman','monk','actors','salesmen',
    'boyfriend','councilman','fella','statesman','paternal','chap','landlord','brethren','lords',
    'blokes','fraternity','bellboy','duke','ballet_dancer','dudes','fiance','colts',
    'husbands','suitor','maternity','he','businessman','masseurs',
    'hero','deer','busboys','boyfriends','kings','brothers','masters','stepfather','brides',
    'son','studs','cowboy','mentleman','sons','baritone','salesman','paramour','male_host',
    'monks','menservants',"mr.",'headmasters','lads','congressman','airman','househusband',
    'priest','barmen','barons','abbots','handyman','beard','fraternities','stewards','colt',
    'czar','stepsons','himself','boys','lions','gentleman','his','masseur','bulls','uncles','bloke','beards',
    'hubby','lion','sorcerer','macho','father','gays','male','waiters','stepson','prostatic_utricle',
    'businessmen','heir','waiter','headmaster','man','governor','god','bridegroom','grandpa',
    'groom','dude','gay','gents','boy','grandfather','gelding','paternity',
    'roosters','prostatic_utricle','priests','manservants','stailor','busboy','heros'

}
#yusu added special
SPECIAL_MALE_NOUNS = {
    'Sperm','Beard','Mustache',
    'Ejaculation','Erection','Scrotum','Penis','Testicles','Epididymis'
}

#yusu added words from the rule file
DEFAULT_FEMALE_NOUNS = {
    'woman', 'women', 'ladies', 'female', 'females', 'girl', 'girlfriend',
    'girlfriends', 'girls', 'her', 'hers', 'lady', 'she', 'wife', 'wives','countrywoman',
'sororal','witches','maidservant','mothers','diva','actress','spinster','mama',
    'duchesses','barwoman','countrywomen','dowry','hostesses','airwomen','princess',
    'governesses','abbess','women','widow','ladies','sorceresses','madam','brides',
    'baroness','housewives','godesses','niece','widows','lady',
'sister','brides','nun','adultresses','obstetrics','bellgirls','her','marchioness',
    'princesses','empresses','mare','chairwoman','convent','priestesses','girlhood',
    'ladies','queen','gals','mommies','maid','female_ejaculation','spokeswoman','seamstress',
    'cowgirls','chick','spinsters','empress','mommy','feminism','gals','enchantress','gal','motherhood','estrogen'
    'camerawomen','godmother','strongwoman','goddess','matriarch','aunt','chairwomen',"ma'am",
    'sisterhood','hostess','estradiol','wife','mom','stewardess','females','viagra',
    'spokeswomen','ma','belle','minx','maiden','witch','miss','nieces','mothered',
    'cow','belles','councilwomen','landlords','granddaughter','fiancees','stepmothers',
    'horsemen','grandmothers','adultress','schoolgirl','hen','granddaughters','bachelorette',
    'camerawoman','moms','her','mistress','lass','policewoman','nun','actresses',
    'saleswomen','girlfriend','councilwoman','lady','stateswoman','maternal',
    'lass','landlady','sistren','ladies','wenches','sorority','bellgirl','duchess',
    'ballerina','chicks','fiancee','fillies','wives','suitress','paternity','she',
    'businesswoman','masseuses','heroine','doe','busgirls','girlfriends','queens',
    'sisters','mistresses','stepmother','grooms','daughter','minxes','cowgirl',
    'lady','daughters','mezzo','saleswoman','mistress','hostess','nuns','maids',"mrs.",
    'headmistresses','lasses','congresswoman','airwoman','housewife','priestess','barwomen',
    'barnoesses','abbesses','handywoman','toque','sororities','stewardesses',
    'filly','czarina','stepdaughters','herself','girls','lionesses','lady','vagina','hers','masseuse','cows','aunts',
    'wench','toques','wife','lioness','sorceress','effeminate','mother','lesbians',
    'female','waitresses','skene_gland','stepdaughter','womb','businesswomen',
    'heiress','waitress','headmistress','woman','governess','godess','bride','grandma','bride','gal','lesbian','ladies','girl',
    'grandmother','mare','maternity','hens','nuns','maidservants','seamstress','busgirl','heroines'

}

SPECIAL_FEMALE_NOUNS = {
    'dowry',
    'womb','pregnant','pregnancy','breast','abortion','miscarriage','Clitoris',
    'menopause','Vagina','Uterus','Ovaries'
}

DEFAULT_TARGET_POS = {'VERB', 'ADJ', 'ADV'}


def get_sentence_gender_cooccurrences(sent, target_pos=None, female_nouns=None, male_nouns=None,female_special=None, male_special=None):
    """
    Get gender cooccurrences for a sentence
    """
    # Handle defaults
    if not target_pos:
        target_pos = DEFAULT_TARGET_POS
    if not female_nouns:
        female_nouns = DEFAULT_FEMALE_NOUNS
    if not male_nouns:
        male_nouns = DEFAULT_MALE_NOUNS
        #yusu's edit
    if not female_special:
        female_special = SPECIAL_FEMALE_NOUNS
    if not male_special:
        male_special = SPECIAL_MALE_NOUNS
        
    gendered_nouns = female_nouns | male_nouns
    special_nouns = female_special | male_special
    #yusu's edit done

    female_cooccur = Counter()
    male_cooccur = Counter()
    #yusu's edit
    female_special_cooccur = Counter()
    male_special_cooccur = Counter()
#yusu's edit done
    # Check which genders are present in this sentence
    female = False
    male = False
    female_s = False
    male_s = False
    for w in sent:
        if w in female_nouns:
            female = True
        elif w in male_nouns:
            male = True
      #yusu's edit 
        elif w in female_special:
            female_s = True
        elif w in male_special:
            male_s = True
          #yusu's edit done
        if female and male:
            break

    # If no gendered words were found, return
    #yusu's edit
    if not (female or male or female_s or male_s):
        return female_cooccur, male_cooccur, female_special_cooccur, male_special_cooccur
#yusu's edit done

    # Find all of the cooccurences of each gender
    for w in en(' '.join(sent)):
        if (w.pos_ in target_pos) and (w.text not in gendered_nouns) and (w.text not in special_nouns):
            item = (w.text, w.pos_)
            if female:
                female_cooccur[item] += 1

            if male:
                male_cooccur[item] += 1
            
            if female_s:
                female_special_cooccur[item] += 1
            
            if male_s:
                male_special_cooccur[item] += 1
                
    return female_cooccur, male_cooccur,female_special_cooccur, male_special_cooccur


def get_sentence_list_gender_cooccurrences(sentences, target_pos=None,
                                           female_nouns=None, male_nouns=None,female_special=None,male_special=None):
    """
    Get gender cooccurrences for a list of sentences
    """
    female_cooccur = Counter()
    male_cooccur = Counter()
#yusu's edit
    female_special_cooccur = Counter()
    male_special_cooccur = Counter()

    for sent in sentences:
        female_cooccur_sent, male_cooccur_sent,female_special_cooccur_sent,male_special_cooccur_sent \
            = get_sentence_gender_cooccurrences(sent, target_pos=target_pos,
                                                female_nouns=female_nouns,
                                                male_nouns=male_nouns,
                                               female_special=female_special,
                                               male_special=male_special)

        female_cooccur += female_cooccur_sent
        male_cooccur += male_cooccur_sent
        female_special_cooccur += female_special_cooccur_sent
        male_special_cooccur += male_special_cooccur_sent

    return female_cooccur, male_cooccur, female_special_cooccur, male_special_cooccur
#yusu's edit done

def get_dataset_gender_cooccurrences(data_dir, vocab, target_pos=None,
                                     female_nouns=None, male_nouns=None,
                                    female_special=None, male_special=None):
    """
    Get gender cooccurrences for a preprocessed dataset
    """
    if type(vocab) == str:
        # Load vocab if a path was provided
        vocab = read_vocab(vocab)

    female_cooccur = Counter()
    male_cooccur = Counter()
    female_special_cooccur = Counter()
    male_special_cooccur = Counter()

    for fname in os.listdir(data_dir):
        fpath = os.path.join(data_dir, fname)
        # Read file
        sentences = read_preprocessed_file(fpath, vocab)
        female_cooccur_file, male_cooccur_file, female_special_cooccur_file, male_special_cooccur_file \
            = get_sentence_list_gender_cooccurrences(sentences, target_pos=target_pos)

        female_cooccur += female_cooccur_file
        male_cooccur += male_cooccur_file
        female_special_cooccur += female_special_cooccur_file
        male_special_cooccur += male_special_cooccur_file

    return female_cooccur, male_cooccur, female_special_cooccur, male_special_cooccur


def compute_gender_cooccurrance_bias(female_cooccur, male_cooccur,female_special_cooccur,male_special_cooccur):
    """
    Compute a bias metric similar to that in (Zhao et al. 2017)
    """
    bias_sum = 0.0
    bias_norm_sum = 0.0
    preserve_sum = 0.0
    preserve_norm_sum = 0.0
    total_words_bias = (set(female_cooccur.keys()) | set(male_cooccur.keys()))
    total_words_preserve = (set(female_special_cooccur.keys()) | set(male_special_cooccur.keys()))

    male_count = sum(male_cooccur.values())
    female_count = sum(female_cooccur.values())
    male_special_count = sum(male_special_cooccur.values())
    female_special_count = sum(female_special_cooccur.values())

    for k in total_words_bias:
        p_w_given_male = male_cooccur[k] / male_count
        p_w_given_female = female_cooccur[k] / female_count

        # Ratio of joint probabilities
        # yusu's edit
        bias_sum += abs((male_cooccur[k]-female_cooccur[k]) / (female_cooccur[k] + male_cooccur[k]))

        # Ratio of conditional probabilities (conditioned on gender)
        bias_norm_sum += p_w_given_male / (p_w_given_male + p_w_given_female)

    bias = bias_sum / len(total_words_bias)
    bias_norm = bias_norm_sum / len(total_words_bias)

    for j in total_words_preserve:
        p_w_given_male_s = male_special_cooccur[j] / male_special_count
        p_w_given_female_s = female_special_cooccur[j] / female_special_count

        # Ratio of joint probabilities
        # yusu's edit
        preserve_sum += abs((male_special_cooccur[k]-female_special_cooccur[k]) / (female_special_cooccur[k] + male_special_cooccur[k]))

        # Ratio of conditional probabilities (conditioned on gender)
        preserve_norm_sum += p_w_given_male_s / (p_w_given_male_s + p_w_given_female_s)

    preserve = preserve_sum / len(total_words_preserve)
    preserve_norm = preserve_norm_sum / len(total_words_preserve)

    return bias, bias_norm, preserve, preserve_norm


def JSD(P, Q):
    """
    Compute Jensen-Shannon divergence
    """
    # Copied from https://stackoverflow.com/a/27432724/1260544
    _P = P / norm(P, ord=1)
    _Q = Q / norm(Q, ord=1)
    _M = 0.5 * (_P + _Q)
    return 0.5 * (entropy(_P, _M) + entropy(_Q, _M))


def compute_gender_distribution_divergence(female_cooccur, male_cooccur):
    """
    Compute Jensen-Shannon divergence between the word probabilities conditioned
    on male and female
    """
    female_count = sum(female_cooccur.values())
    male_count = sum(male_cooccur.values())
    total_words = (set(female_cooccur.keys()) | set(male_cooccur.keys()))
    

    p_w_given_male = np.array([male_cooccur[k] / male_count for k in total_words])
    p_w_given_female = np.array([female_cooccur[k] / female_count for k in total_words])

    return JSD(p_w_given_female, p_w_given_male)
  
def compute_gender_distribution_divergence_preserve(female_special_cooccur, male_special_cooccur):
    """
    Compute Jensen-Shannon divergence between the word probabilities conditioned
    on male and female
    """
    male_special_count = sum(male_special_cooccur.values())
    female_special_count = sum(female_special_cooccur.values())
    total_words_preserve = (set(female_special_cooccur.keys()) | set(male_special_cooccur.keys()))
    

    p_w_given_male_s = np.array([male_special_cooccur[k] / male_special_count for k in total_words_preserve])
    p_w_given_female_s = np.array([female_special_cooccur[k] / female_special_count for k in total_words_preserve])

    return JSD(p_w_given_female, p_w_given_male)
 